# **Projekt: Fehlererkennung in Getrieben**
## Netztyp 2: FFT-Net

Mey et al. (Mey et al. 2020) untersuchten die automatisierte Erkennung von Unwuchten in rotierenden Wellen mithilfe von Maschinellem Lernen und veröffentlichten den hierbei erstellten Datensatz auf der Plattform Kaggle (Mey et al., 2022). In diesem Kontext entwickeln und analysieren sie vier Klassifizierungsalgorithmen: Convolutional Neural Network (CNN), Fully-Connected Neural Network (FCNN), Hidden Markov Model und Random Forest. Diese Algorithmen unterscheiden zwischen den Zuständen "eine Unwucht liegt vor" und "keine Unwucht liegt vor". Das FCNN erreicht hier, als die erfolgreichste Methode, eine Vorhersagegenauigkeit von 98,5 %. <br>
Das vorliegende Projekt reproduziert zunächst das von Mey et al. beschriebene CNN. Auf Basis des FCNN wird zudem ein Neuronales Netzwerk (NN) zur binären Klassifizierung (Binary FFT-Net) und zur Multiklassen-Klassifizierung (Multiclass FFT-Net) entwickelt. Das Modell zur Multiklassen-Klassifizierung unterscheidet die im Datensatz angegebenen fünf Unwucht-Stufen. Das CNN verarbeitet hierbei direkt Sensordaten. Für das FCNN überführt die Fast Fourier Transform (FFT) die Daten vorab in den Frequenzbereich. <br>
Im Rahmen des Projektes sind so mehrere Python-Module entstanden. Der vollständige Code ist in einem GitHub-Repository (Kuhlen et al., 2024) verfügbar.


<details>
    <summary open><b>Abkürzungsverzeichnis</b></summary>

> <br>
>
> | Abkürzung | Bedeutung |
> |:----------|:----------|
> | CNN | Convolutional Neural Network |
> | CSV | Comma-Separated Values |
> | DFT | Discrete Fourier-Transform, Diskrete Fourier-Transformation |
> | FCNN | Fully Connected Neural Network |
> | FFT | Fast Fourier Transform |
> | IQR | Inter-Quantile Range, Interquantilabstand |
> | NN | Neural Network, Neuronales Netzwerk |
> | UUID | Universally Unique Identifier |
> | YAML | Yet Another Markdown Language |
>
> <br>
</details>


### **Teil B.1: Binary FFT-Net**

#### **Die Projektstruktur**

Das FFT-Net gliedert sich in die drei Jupyter Notebooks *fft_net_preprocessing.ipynb*, *fft_net_biclassification.ipynb* und *fft_net.ipynb*. Die *fft_net_preprocessing.ipynb* umfasst den Download des Datensatzes und das eigentliche Preprocessing. Das *fft_net_biclassification.ipynb* und *fft_net.ipynb* laden die aufbereiteten Daten ein. Sie erstellen, trainieren und bewerten das jeweilige NN. Im Projekt finden sich diese Skripte im Ordner *working_dir/notebooks/*. Auf die Nennung des Working Directories *working_dir* wird in nachfolgenden Dateipfaden verzichtet. <br>
Auf dem Pfad *src/* befinden sich die zwei Python-Skripte *data_loader.py* und *input_preparation.py* sowie die Ordner *net_models*, *preprocessing/* und *visualization.py*. Jedes dieser Module umfasst Funktionen für einen spezifischen Prozessschritt hin zu einem funktionsfähigen NN. Sie sind im Folgenden in chronologischer Reihenfolge beschrieben. <br>
Das Modul `data_loader` umfasst Funktionen zum Herunterladen des Kaggle-Datensatzes sowie zum Laden und Speichern von Datensätzen aus der und in die Projekt-Ordnerstruktur. Im Modul `preprocessing` sind alle Funktionen für die Vorverarbeitung der Daten enthalten. Die Vorbereitung der vom Preprocessing aufbereiteten Daten auf die Eingabe in das Neuronale Netz übernehmen Funktionen aus der `input_preparation`. Konstruktoren für die FFT-Net-Varianten sind im Modul `net_models` enthalten. Hier sind zudem weitere Funktionen mit Bezug auf das Trainieren und die Performance-Analyse von NN abgelegt. Das Modul `visualization` bietet Funktionen zur Visualisierung von Daten über den gesamten Prozess.

In [ ]:
# import libraries
from uuid import uuid4
import os
from pathlib import Path

# module imports from /src
if Path.cwd().stem == "notebooks":
    os.chdir( Path.cwd().parent)

import src.data_loader as dl
import src.visualization as vis
import src.net_models as net
import src.input_preparation as ip

Zu Beginn des Codes erlauben einige Booleans die Kontrolle des Prozessablaufs. Über einen hier gesetzten `CONTROL_INDEX` erfolgt zu Beginn des Prozesses die stichprobenartige Visualisierung der Daten. An dieser Stelle wird zudem ein Universally Unique Identifier (`UUID`) generiert, der bei der Speicherung der Netzparameter und der im Verlauf generierten Graphen als Ordnername verwendet wird. <br>

In [ ]:
# set the source directory for the preprocessed data to use
SOURCE: str = "f2fcf2aa-bd96-4d89-8bb5-4a0a1bc11b1b"

# setup a single index for control purposes
CONTROL_INDEX: int = 0

# generate a unique identifier for this run
UUID: str = str(uuid4())
print(f"Current UUID:\t{UUID}")

In [ ]:
# setup system and check the number of cpu cores and gpus available
net.system_setup()

#### **Laden der Datensätze**
Üblicherweise werden für das Training eines neuronalen Netzes mittels Supervised Learning die Daten in mindestens zwei separate Datensätze aufgeteilt: einen Trainings- und einen Testdatensatz. Mit dem Testdatensatz können nach dem Training zukünftige (dem Netz unbekannte) Dateneingaben simuliert werden und so vorab die Zuverlässigkeit des Trainings beurteilt werden (Liu, 2025, S. 39).
Nach dem Laden der Daten aus dem *data*-Ordner werden CSV-Dateien mit der Endung *D* in der Liste *development_data* und Dateien mit der Endung *E* in der Liste *evaluation_data* gespeichert.

In [ ]:
# load data from folder and split in training and evaluation data
data_path = Path().cwd() / "data" / "processed" / SOURCE
development_data, evaluation_data = dl.load_all_datasets(data_path)

# define file path for metrics and confusion matrix
figure_path = Path().cwd() / "results" / "metrics" / ("b_" + UUID)

Exemplarisch wird ein Datensatz aus den Development-Daten in mehreren Subplots visualisiert, um sicherzustellen, dass die Daten korrekt geladen wurden. 

In [ ]:
# visualize one random dataset for data validation
_ = vis.plot_column_data(development_data[CONTROL_INDEX],
                             development_data[CONTROL_INDEX].columns)

#### **Vorbereitung der Daten zur Eingabe in das neuronale Netz**

Um das neuronale Netz trainieren zu können, ist eine Aufbereitung der Trainings- und Testdaten erforderlich.
Dazu werden die einzelnen DataFrames mit den Trainingsdaten zu einem einheitlichen DataFrame zusammengeführt. Gleichzeitig wird eine zusätzliche Spalte *Label* erstellt, welche die gemessenen Unwuchtstärken enthält und somit als *Output* für die spätere Klassifizierung dient (Liu, 2025, S. 19). Die Labels werden in das One-Hot-Encoding-Format überführt, das für die Multiklassen-Klassifizierung in Deep-Learning-Systemen benötigt wird (Liu, 2025, S. 74).
Anschließend werden die DataFrame-Spalten mit den Amplituden der Vibrationen als *Input* (Features) für das NN spezifiziert.

In [ ]:
full_development_dataset = ip.concatenate_datasets(development_data, use_binary_labeling=True)

training_samples_dict = ip.split_data(
    dataframe=full_development_dataset,
    data_columns=[
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

Mit dem Aufruf der Funktion `check_data` werden die im vorbereiteten Datensatz enthaltenen Klassen sowie deren jeweilige Häufigkeiten überprüft, um einen Überblick über die Verteilung der verschiedenen Labels im Trainingsdatensatz zu gewinnen. Es ist wichtig eine ausgewogene Klassen-Verteilung sicherzustellen, da eine ungleiche Verteilung zu Bias und damit zu ungenauen Vorhersagen führen kann (Liu, 2025, S.36). <br>
Die Ausgabe von `check_data` zeigt, dass alle fünf Klassen in etwa 6430 Einträge umfassen und somit einen ausgeglichenen Querschnitt der Trainingsdaten abbilden. 

In [ ]:
ip.check_data(training_samples_dict)

print(training_samples_dict["samples"].shape)

#### **Das Neuronale Netz**

Im nachfolgenden Codeabschnitt werden alle relevanten Parameter und Hyperparameter, die für den Aufbau und das Training des NN benötigt werden, zentral definiert. Durch die Zusammenfassung an einer Stelle bleibt der Code übersichtlicher und Anpassungen am Lernverhalten des Modells können schnell vorgenommen werden. <br>
Die Konstanten sind nach ihrer Zuständigkeit in separate Abschnitte zur Modellerstellung, Modellkompilierung und zum Training des Modells gegliedert. Abschließend werden in dem Dictionary `training_samples_dict` Gewichtungen auf die Unwuchtklassen individuell eingestellt. Diese Gewichtungen können genutzt werden, wenn einzelne Klassen im Datensatz unterrepräsentiert sind oder vom Modell schlechter erkannt werden, um für eine ausgewogenere Vorhersageleistung zu sorgen.

In [ ]:
# model creation parameters
N_HIDDEN_LAYERS: int = 2
L2: float = 2e-2 # 0.02
DROPOUT: float = 0.2
NEGATIVE_SLOPE: float = 0.01 #0.3

# model compilation parameters
LEARNING_RATE: float = 1e-6

# model training parameters
BATCH_SIZE: int = 128
EPOCHS: int = 200
VALIDATION_SPLIT: float = 0.1

# manual changes to class weights
training_samples_dict["class_weights"] = {
    0: 25703 /(25703 + 6428),  # none (Klasse 0) wird höher gewichtet
    1: 6428 / (25703 + 6428),  # some (Klasse 1) wird weniger gewichtet
}

In der Methode `binary_classifier.build_model` wird die *Keras Sequential API* eingesetzt, um ein NN als linearen Stapel von Layern zu definieren (Keras Developers, o. D.-c). Hierfür wird eine Instanz der Klasse Sequential erzeugt und mit Input-, Output- sowie der zuvor spezifizierten Anzahl Hidden Layers erweitert.
Analog zu Mey et al. umfasst das erste Dense-Layer 2048 Neuronen. In diesem Projekt ist dieses Layer um eine *L2-Regularisierung* sowie ein *Dropout* ergänzt, um Overfitting vorzubeugen (vgl. Liu, 2025, S.40f & 194). <br>
Als Aktivierungsfunktion kommt eine *LeakyReLU* mit einstellbarem *negative_slope* zum Einsatz, um das sogenannte "Dying ReLU Problem", also die langfristige Deaktivierung einzelner Neuronen, zu vermeiden (Adari & Alla, 2024, S.200). Abschließend wird im Output-Layer eine *Softmax*-Aktivierungsfunktion genutzt, um damit das Modell auf eine Multiklassen-Klassifikation anzupassen. Die Softmax-Funktion wird für Klassifikationsaufgaben verwendet, bei denen eine spezifische Klasse aus dem Input vorhergesagt werden soll (Adari & Alla, 2024, S.203).

In [ ]:
model = net.binary_classifier.build_model(n_hidden_layers=N_HIDDEN_LAYERS,
                                   training_samples_dict=training_samples_dict,
                                   l2=L2,
                                   dropout=DROPOUT,
                                   negative_slope=NEGATIVE_SLOPE)

Im anschließenden Schritt wird das zuvor definierte Modell mithilfe der Funktion `compile` für das Training vorbereitet. Über den Parameter `learning_rate` wird festgelegt, wie stark die Gewichte des Netzes bei jedem Update angepasst werden (vgl. Adari & Alla, 2024, S.216).

In [ ]:
net.binary_classifier.compile(model=model, learning_rate=LEARNING_RATE)

Das Training des Modells wird mit der Funktion `train` aufgerufen. Dabei werden die Trainingsdaten aus dem *samples_dict*, die maximale Epochenanzahl, die *Batch-Size* und der *Validation-Split* als Parameter übergeben. Zusätzlich kann ein *Early Stopping*-Mechanismus konfiguriert werden. <br>
Early Stopping überwacht die Validierungsverluste und beendet das Training vorzeitig, wenn sich der Verlust über mehrere Epochen nicht mehr verbessert. Somit muss das Training bei suboptimal getroffenen Einstellungen des NNs nicht vollständig durchlaufen und wird vorzeitig abgebrochen. Das spart Rechenleistung und kann Overfitting vermeiden (vgl. Adari & Alla, 2024, S. 225). Die Implementierung erfolgt über den EarlyStopping-Callback von Keras (Keras Developers, o. D.-a). In der Funktion `train` ist EarlyStopping standardmäßig aktiviert, kann jedoch über den Parameter `use_early_stopping` bei Bedarf deaktiviert werden. <br>
Das Training selbst wird über die übergebene Anzahl von Epochen mit der Methode `model.fit` durchgeführt (Keras Developers, o. D.-b). Der Parameter `shuffle=True` sorgt dafür, dass die Trainingsdaten vor jeder Epoche durchmischt werden, wodurch das Modell unabhängiger von der Reihenfolge der Daten wird. Mit dem Parameter `class_weight` können die Gewichtungen der einzelnen Klassen gesetzt werden, um das Training bei unausgeglichenen Datensätzen zu verbessern (Keras Developers, o. D.-b). <br>
Am Ende gibt die Funktion ein *keras.callbacks.History*-Objekt zurück. Dieses Objekt speichert Metriken wie *Loss*, *Accuracy*, *Precision* und *Recall* über den Trainingsverlauf.

In [ ]:
history = net.train(
    model=model,
    samples_dict=training_samples_dict,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    use_early_stopping=True,
)

#### **Bewertung des Neuronalen Netzes**

Im Anschluss an das Training werden die Daten aus dem History-Objekt visualisiert. Die Funktion `plot_history` ermöglicht es, den Verlauf von Metriken über die Trainingsepochen hinweg grafisch darzustellen. <br>
Anhand der Loss Kurven von Trainings- und Validierungs-Daten lässt sich überprüfen, ob das Modell Overfitting-Verhalten aufweist. Gemäß Adari und Alla liegt ein typischer Overfitting-Fall vor, wenn das Trainings-Loss kontinuierlich sinkt, während das Validierungs-Loss ab einer gewissen Epoche beginnt zu steigen (2024, S. 225). Idealerweise sollten sich beide Kurven gleichermaßen einem geringen Wert annähern. Nach dem Training mit 20 Epochen, nähert sich das Training-Loss dem Wert 0.2 und das Validation-Loss dem Wert 0.03 an. Die zwei Kurven liegen dicht beieinander und somit weist das Training kein Overfitting-Verhalten auf.<br>
Bezüglich Accuracy verweisen Adari und Alla darauf, dass diese zwar eine erste Aussage über die Prognosefähigkeit des Modells liefert (2024, S. 107), jedoch in vielen Fällen – insbesondere bei Klassifikationsaufgaben – nicht ausreichend aussagekräftig ist (2024, S. 115). Deshalb empfiehlt Liu Precision und Recall in die Modellevaluation miteinzubeziehen (2025, S. 300). Precision gibt den prozentualen Anteil korrekter Positivvorhersagen am Gesamtvolumen an, während Recall den Prozentsatz tatsächlich erkannter Positiver innerhalb aller Positiven beschreibt (Adari & Alla, 2024, S. 116). <br>
Adari und Alla definieren die F1-Metrik als harmonischen Mittelwert von Precision und Recall, bei dem ein höherer Wert darauf hinweist, dass beide Größen vergleichsweise hoch sind (2024, S. 116). Liu beschreibt, dass ein steigender F1-Wert auf eine höhere Prognosekraft des Modells schließen lässt: Ein Wert nahe 1 deutet auf ein nahezu perfektes Klassifikationsmodell hin, während ein F1-Wert nahe 0 auf eine sehr geringe Vorhersagefähigkeit schließen lässt (2025, S. 414).

In [ ]:
# visualize training metrics
figure_dicts = vis.plot_history(
    history,
    metrics=["accuracy", "f1_score", "loss", "precision", "recall"],
    separated=True,
)

for figure_dict in figure_dicts:
    dl.save_figure(
        parent_path=figure_path,
        figure_dict=figure_dict,
        format="png",
        dpi=300,
    )

Um das Modell auf unbekannte Daten zu testen, wird der Evaluationsdatensatz analog zur bereits beschriebenen Trainingsdaten-Aufbereitung (vgl. Abschnitt „Vorbereitung der Daten zur Eingabe in das neuronale Netz“) mithilfe der Funktionen `concatenate_datasets` und `split_data` strukturiert.

In [ ]:
full_evaluation_dataset = ip.concatenate_datasets(
    evaluation_data, use_binary_labeling=True
)

test_samples_dict = ip.split_data(
    dataframe=full_evaluation_dataset,
    data_columns=[
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

Folgend wird die Funktion `check_data` auf den Evaluationsdatensatz angewendet, um die vorhandenen Klassenverteilungen zu überprüfen. Aus den One-Hot-kodierten Labels wird dabei abgeleitet, wie viele Samples pro Klasse im Evaluationsdatensatz enthalten sind. Die Ausgabe zeigt, dass auch die Evaluationsdaten eine ausgeglichene Klassenverteilung mit etwa 1669 Einträgen pro Klasse aufweisen und diese dementsprechend mit den Trainingsdaten verglichen werden können.

In [ ]:
ip.check_data(test_samples_dict)

Gemäß der offiziellen Dokumentation von Keras wird mithilfe der Methode `evaluate`  im Testmodus eine Vorhersage auf dem unbekannten Testdatensatz durchgeführt. Die Methode berechnet den Loss als auch die zuvor definierten Metriken (vgl. Keras Developers, o. D.-b). Anschließend werden diese Werte in der Ausgabe angezeigt. In diesem Projekt wird `evaluate` über eine gleichnamige Wrapper-Funktion aufgerufen und erhält die Testdaten als Eingabe. Damit lässt sich untersuchen, ob das Modell – neben den guten Resultaten auf den Trainingsdaten – auch auf neuen Datensätzen eine zuverlässige Klassifikation der Unwuchtklassen liefert. Nach Adari und Alla dient dieser Evaluationsschritt als aussagekräftiger Indikator für die Wirksamkeit des Modells (2024, S. 109).

In [ ]:
# evaluate model
evaluation = net.evaluate(model=model, test_samples_dict=test_samples_dict, batch_size=BATCH_SIZE)
display(evaluation)

##### **Confusion Matrix**
Um die Qualität der Klassifikationsergebnisse zu prüfen wird abschließend eine Confusion Matrix erstellt. Diese baut auf den predictions der Keras `predict` Methode auf, welche Vorhersagen für die Testdaten generiert (Keras Developers, o. D.-b). Die gleichnamige Wrapperfunktion gibt die tatsächlichen und die vorhergesagten Klassen zurück. Die Funktion `plot_confusion_matrix` erstellt eine Matrix mit den tatsächlichen Klassen in den Zeilen und den vorhergesagten Klassen in den Spalten. Auf der Diagonalen von oben links nach unten rechts sind die korrekten Vorhersagen abzulesen. Je dunkler die Farbe der Kachel ist, desto mehr Vorhersagen wurden korrekt getroffen.

In [ ]:
# predictions by model
true_labels, predicted_labels = net.predict(
    model=model, test_samples_dict=test_samples_dict
)

print(type(test_samples_dict["encoder"].classes_))

# plot confusion matrix
figure_dict = vis.plot_confusion_matrix(
    true_labels=true_labels,
    predicted_labels=predicted_labels,
    class_names=test_samples_dict["encoder"].classes_.tolist(),
)


dl.save_figure(
    parent_path=figure_path,
    figure_dict=figure_dict,
    format="png",
    dpi=300,
)

dl.save_model_hyperparameters(
    parent_path=figure_path,
    model=model,
    learning_rate = LEARNING_RATE,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
)

#### **Quellen**

##### **Literatur**

Adari, S. K. & Alla, S. (2024). Beginning Anomaly Detection Using Python-Based Deep Learning. https://doi.org/10.1007/979-8-8688-0008-5

Harris, C. R., Millman, K. J., Van der Walt, S. J., Gommers, R., Virtanen, P., Cournapeau, D., Wieser, E., Taylor, J., Berg, S., Smith, N. J., Kern, R., Picus, M., Hoyer, S., Van Kerkwijk, M. H., Brett, M., Haldane, A., Del Río, J. F., Wiebe, M., Peterson, P., . . . Oliphant, T. E. (2020). Array programming with NumPy. Nature, 585(7825), 357–362. https://doi.org/10.1038/s41586-020-2649-2

Liu, Z. (2025). Deep Learning. In: Artificial Intelligence for Engineers. Springer, Cham. https://doi.org/10.1007/978-3-031-75953-6_8

Mey, O., Neudeck, W., Schneider, A. & Enge-Rosenblatt, O. (2020). Machine learning-based unbalance detection of a rotating shaft using vibration data. In 2020 25th IEEE International Conference on Emerging Technologies and Factory Automation (ETFA) (S. 1610–1617). IEEE. https://doi.org/10.1109/etfa46521.2020.9212000

Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M. & Duchesnay, É. (2011). Scikit-learn: Machine Learning in Python. Journal Of Machine Learning Research, 12, 2825–2830. https://doi.org/10.5555/1953048.2078195


##### **Online**
Kuhlen, N., Beißner, L., Bergermann, T. & Wallrad, L. (2024). AI Gear Fault Detection (Version V1) [Computer Software]. https://github.com/ninakuhlen/ai-gear-fault-detection

Keras Developers. (o. D.-a). Keras documentation: EarlyStopping. https://keras.io/api/callbacks/early_stopping/

Keras Developers. (o. D.-b). Keras documentation: Model training APIs. https://keras.io/api/models/model_training_apis/

Keras Developers. (o. D.-c). Keras documentation: The Sequential class. https://keras.io/api/models/sequential/

Mey, O., Neudeck, W., Schneider, A. & Enge-Rosenblatt, O. (2022, 23. Februar). Vibration Analysis on Rotating Shaft. Kaggle. Abgerufen am 20. Oktober 2024, von https://www.kaggle.com/datasets/jishnukoliyadan/vibration-analysis-on-rotating-shaft

NumPy Developers. (o. D.). Discrete Fourier Transform (Numpy.FFT) — NUMPY v2.2 Manual. https://numpy.org/doc/stable/reference/routines.fft.html#module-numpy.fft

Scikit-learn Developers. (o. D.). RobustScaler. Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html